# TASHO - A model predictive control toolchain for constraint-based task specification of robot motions

#### Presented by: Alejandro Astudillo and Ajay Sathya

Tasho stands for “**TA**sk **S**pecification with receding **HO**rizon control”


First, we need to import the relevant modules from Tasho and other dependencies (casadi, numpy)

In [1]:
import os
os.chdir(str(os.getcwd())+"/../../")


from tasho import robot as rob

from tasho import problem_template as pt
from tasho import environment as env

import casadi as cs
from rockit import MultipleShooting, Ocp
import numpy as np

## Set robot

In [2]:
# Initial joint configuration
q0_val = [0, -0.523598, 0, 2.51799, 0, -0.523598, 0]
# Initial joint velocity
q_dot0_val = [0]*7

robot = rob.Robot('kinova')

robot.set_joint_acceleration_limits(lb = -30*3.14159/180, ub = 30*3.14159/180)

robot.set_state(q0_val + q_dot0_val)

robot.set_robot_input_resolution("acceleration")

Loading robot params from json ...
Loaded 7-DoF robot: kinova


## Set environment

In [3]:
environment = env.Environment()

cube1 = env.Cube(length = 1, position = [0.5, 0, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/cube_small.urdf")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf = "/models/objects/table.urdf")
#table2 = env.Box(height = 0.3, position = [0,0.5, 0], orientation = [0.0, 0.0, 0.0, 1.0], urdf = "/models/objects/table.urdf")

In [4]:
environment.add_object(cube1, "cube")
environment.add_object(table1, "table1")
#environment.add_object(table2, "table2")

In [5]:
environment.print_objects()

['cube', 'table1']


## Task: Approximation to object

In [6]:
horizon_size = 10
t_mpc = 0.5

T_goal = np.array([[0, 1, 0, 0.5], [1, 0, 0, 0], [0, 0, -1, 0.25], [0, 0, 0, 1]])  # T_goal = np.array([[0.0, 0., -1., 0.5], [0., 1., 0., 0.], [1.0, 0., 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0., 0., -1., 0.5], [-1., 0., 0., 0.], [0., 1., 0.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0., 1., 0., 0.5], [1., 0., 0., 0.], [0., 0., -1.0, 0.5], [0.0, 0.0, 0.0, 1.0]]) # T_goal = np.array([[0, 1, 0, 0], [1, 0, 0, -0.5], [0, 0, -1, 0.5], [0, 0, 0, 1]])

approach_task = pt.Point2Point(horizon_size*t_mpc, horizon_steps = horizon_size, goal = T_goal)

approach_task.add_robot(robot)

Goal transformation matrix


### Set parameter values

In [7]:
approach_task.ocp.set_value( approach_task.parameters["q0"], q0_val)
approach_task.ocp.set_value( approach_task.parameters["q_dot0"], q_dot0_val)

### Solve task

In [8]:
sol = approach_task.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      530
Number of nonzeros in inequality constraint Jacobian.:      234
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      225
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa


## Simulate plan execution


In [9]:
from tasho import world_simulator
import pybullet as p

obj = world_simulator.world_simulator()

position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position, orientation, 'kinova')
joint_indices = [0, 1, 2, 3, 4, 5, 6]

environment.set_in_world_simulator(obj)
cubeID = obj.objectIDs[0]

no_samples = int(t_mpc/obj.physics_ts)

obj.resetJointState(robotID, joint_indices, q0_val)
obj.setController(robotID, "velocity", joint_indices, targetVelocities = [0]*robot.ndof)
obj.run_simulation(250) # Here, the robot is just waiting to start the task

# Sample the solution for the next MPC execution
ts, q_sol = sol.sample(approach_task.states["q"], grid="control")
ts, qdot_sol = sol.sample(approach_task.states["q_dot"], grid="control")
horizon_size = int(qdot_sol.size/robot.ndof - 1)

for i in range(horizon_size):
    q_vel_current = 0.5*(qdot_sol[i] + qdot_sol[i+1])
    obj.setController(robotID, "velocity", joint_indices, targetVelocities = q_vel_current)
    obj.run_simulation(no_samples)


obj.run_simulation(100)

obj.end_simulation()

Ending simulation
